# Closed-Form Solutions to Multiple-View Homography Estimation
This noteboook contains the implementation of the graph synchronization method that can be used to solve the stitching problem in case of multiple images. The implementation resamble the procedure explained in Shroeder et al. 2011.

In [1]:
import cv2 as cv
import os
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import networkx as nx

## Variables definition

In [2]:
#Directories
dataset_name = 'small_dataset'
dataset_dir = f'datasets/{dataset_name}'
sift_dir = 'sift'
matches_dir = 'matches'
salient_matches_dir ="salient_matches"
output_dir ="output"

In [3]:
#If true store the computed images on disk
save_output = True
G_filename = f"G_{dataset_name}.npy" #Name of the file where matrix G will be saved
gamma_filename = f"gamma_{dataset_name}.npy"
weight_matrix_filename = f"w_{dataset_name}.npy"

## Classes and functions definition

In [4]:
#This function loads images from a specific folder
def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv.imread(os.path.join(folder,filename))
        img = cv.cvtColor(img, cv.COLOR_BGR2RGB)
        if img is not None:
            images.append(img)
    return images

In [5]:
#This class contains the methods and properties related to the features of an image
class ImageFeature:
    def __init__(self, image, index):
        self.image = image
        self.index = index
    
    #Compute the salient points of the image using SIFT algorithm
    def SIFT(self, save_output = False, output_dir = ""):
        sift = cv.SIFT_create()
        self.kp, self.des = sift.detectAndCompute(self.image,None)
        self.img_with_sift=cv.drawKeypoints(self.image,self.kp,self.image)
        if save_output:
            cv.imwrite(os.path.join(output_dir,f"{self.index+1}.jpg"), self.img_with_sift)

In [6]:
#This class contains the methods and properties related to the problem of feature matching between two images
class Match:
    def __init__(self, image_feature_source, image_feature_destination):
        self.image_feature_source = image_feature_source
        self.image_feature_destination = image_feature_destination
    
    #This function performs feature matching between to images
    def feature_matching(self, threshold, save_output = False, output_dir = ""):
        # FLANN parameters
        FLANN_INDEX_KDTREE = 1
        index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
        search_params = dict(checks=50)   # or pass empty dictionary
        flann = cv.FlannBasedMatcher(index_params,search_params)
        matches = flann.knnMatch(self.image_feature_source.des,
                                 self.image_feature_destination.des,
                                 k=2)
        # Need to draw only good matches, so create a mask
        self.good = []
        for m,n in matches:
            if m.distance < threshold*n.distance:
                self.good.append((m,n))
        draw_params = dict(matchColor = (0,255,0),
                           singlePointColor = (255,0,0),
                           flags = cv.DrawMatchesFlags_DEFAULT)
        self.img_with_match = cv.drawMatchesKnn(self.image_feature_source.image,
                                                self.image_feature_source.kp,
                                                self.image_feature_destination.image,
                                                self.image_feature_destination.kp,
                                                self.good,None,**draw_params)
        if save_output:
            cv.imwrite(os.path.join(output_dir,f"match_{self.image_feature_source.index+1}_{ self.image_feature_destination.index+1}.jpg"),self.img_with_match)
    
    #This function allows to check whether the match between the source and destination images is to be considered a good match
    def check_salient(self, threshold, save_output = False, output_dir = ""):
        #A threshold is used to define the goodness of the  match
        if len(self.good) > threshold:
            if save_output:
                cv.imwrite(os.path.join(output_dir,f"salient_{self.image_feature_source.index+1}_{ self.image_feature_destination.index+1}.jpg"),self.img_with_match)
            return True
        return False
    
    def mult_and_norm(self,H, points):
        proj_p = np.dot(H,points)
        proj_p = proj_p / proj_p[2,:]
        return proj_p[0:2,:]
    
    #This function allows to estimate the homography between the two images
    def fit_homography(self):
        self.src_pts = np.float32([self.image_feature_source.kp[m[0].queryIdx].pt for m in self.good])
        self.dst_pts = np.float32([self.image_feature_destination.kp[m[0].trainIdx].pt for m in self.good])
        
        self.src_pts = np.concatenate((self.src_pts, np.ones([self.src_pts.shape[0],1])), axis=1)
        self.dst_pts = np.concatenate((self.dst_pts, np.ones([self.dst_pts.shape[0],1])), axis=1)
        
        rescale = 1./np.max(self.image_feature_source.image.shape)
        T_norm = np.diag([rescale,rescale,1])
        
        #print(T_norm)
        
        #print(self.src_pts[0,:])
        #print(self.src_pts.shape)
        
        src_pts_proj = self.mult_and_norm(np.eye(3),self.src_pts.transpose()).transpose()
        dst_pts_proj = self.mult_and_norm(np.eye(3),self.dst_pts.transpose()).transpose()
        
        #print(src_pts_proj.shape)
        #print(src_pts_proj[0,:])
        
        self.M, self.mask = cv.findHomography(src_pts_proj, dst_pts_proj, cv.RANSAC, 3.0)
        
        self.M = T_norm @ self.M @ np.linalg.inv(T_norm)
    
    #This function allows to normalize the homography so that the determinant is unitary
    #This makes homographies part of the SL(3) group
    def normalize_homography(self):
        det = np.linalg.det(self.M)
        self.H = self.M/np.cbrt(det)
        new_det = np.linalg.det(self.H)
        
    #Remember: if src is image I and dest is image J, we are finding H from I to J, this is 
    #Hj,i
    

## Example

In [7]:
#Load the dataset
imgs = load_images_from_folder(dataset_dir)
n = len(imgs) #Number of images

(<matplotlib.image.AxesImage at 0x1846d2627c0>, None)

In [8]:
#Plot an image as example
figure(figsize=(40, 40), dpi=80)
plt.imshow(imgs[0],),plt.show()

In [9]:
#For each image compute the salient points
image_features =[] #Array that will contain the salient features of each image
for i in range(0,n):
    image_feature = ImageFeature(imgs[i], i)
    image_feature.SIFT(save_output, sift_dir)
    image_features.append(image_feature)

In [10]:
matching_threshold = 0.6

In [11]:
#Compute the good matches between each pair of images
matches =[]
for i in range(0, n-1):
    for j in range(i+1, n):
        match = Match(image_features[i], image_features[j])
        match.feature_matching(matching_threshold, save_output, matches_dir)
        matches.append(match)

matches_th = 10
salient_matches = list(filter(lambda x: x.check_salient(matches_th, save_output, salient_matches_dir),matches))  


In [12]:
#For every good match, compute also the matches in the reverse order (destination, source)
total_matches = []
for match in salient_matches:
    inv_match = Match(match.image_feature_destination, match.image_feature_source)
    inv_match.feature_matching(matching_threshold, save_output, matches_dir)
    total_matches.append(match)
    total_matches.append(inv_match)

In [13]:
weight_matrix = np.zeros([n,n])
for match in total_matches:
    weight_matrix[match.image_feature_source.index, match.image_feature_destination.index]=len(match.good)

## Globally Scaled Homographies (GSH)
Once the good matches have been found, it is possible to used the GSH method in order to get a closed form solution for the problem of aligning all images to a specific coordinate frame.
A homography between two images can easily be estimated from point correspondences using standard procedures. These methods generally ignore the additional information provided by the remaining images in the set and under presence of noise the consistency relationship $H_{i,j} = H_{k,j}*H_{i,k}$ is violated.
This means, that two images may align well to a reference image but may at the same time be badly aligned with each other.In other words, aligning n images to a reference image in order to create a mosaic, requires a set of homographies $U_1,...,U_n$ which by construction
consistently aligns all images to a common reference frame. These global homographies respect the relationship: $H_{i,j} = U_j*U_i^{-1}$.
At this point, the problem is to minimize $C_{GSH}(\hat{U}) = ||G\hat{U}||^2$ under the constraint $\hat{U}^T*\hat{U} = I$. The three right singular vectors associated with the 3 smallest singular values of G provide a solution for the three columns of $\hat{U}$. Thus the problem can be solved using SVD(G). In the previous formulas, $C_{GSH}$ is the cost function to be minimized and G is a matrix computed as follows:
\begin{equation*} G = 
\begin{bmatrix}
    (1-\zeta_1)I & (\gamma H)_{2,1} & \dots  & (\gamma H)_{n,1} \\
    (\gamma H)_{1,2} & (1-\zeta_2)I & \ddots  & \vdots \\
    \vdots & \ddots & \ddots & (\gamma H)_{n,n-1} \\
    (\gamma H)_{1,n} & \dots & (\gamma H)_{n-1,n} & (1-\zeta_n)I
\end{bmatrix}
\end{equation*}
In the matrix, the two terms $\gamma$ and $\zeta$ are defined as follows:
\begin{equation*} \gamma_{i,j} = 
\begin{cases}
1 & \text{if $H_{i,j}$ exists},\\
0 & \text{otherwise}
\end{cases}
\end{equation*}
\begin{equation*} \zeta_{k} = \Sigma_{i = 1}^{n} \gamma_{i,k}
\end{equation*}

In [14]:
#Computation of gamma and zeta for each pair of images
gamma = np.eye(n,n)
for match in total_matches:
    gamma[match.image_feature_destination.index, match.image_feature_source.index] = 1
    
zeta = np.sum(gamma, axis=0)
print(gamma)
print(zeta)

[[1. 1. 1.]
 [1. 1. 1.]
 [1. 1. 1.]]
[3. 3. 3.]


In [15]:
#Computation of a big block matrix H where each block represent an homography between two images
H = np.eye(3*n, 3*n)
for match in total_matches:
    match.fit_homography()
    match.normalize_homography()
    H[3*match.image_feature_destination.index:3*(match.image_feature_destination.index+1),
     3*match.image_feature_source.index:3*(match.image_feature_source.index+1)] = match.H
print(H)

[[ 1.          0.          0.          0.8551021   0.04408762  0.32397736
   1.08865217 -0.02842227 -0.52355885]
 [ 0.          1.          0.         -0.11780556  1.0038624   0.01289516
   0.16009339  0.9470506   0.05760714]
 [ 0.          0.          1.         -0.22870361  0.01659748  1.07291224
   0.34991683 -0.12023143  0.78165858]
 [ 1.0769052  -0.04297181 -0.32430265  1.          0.          0.
   1.05125001 -0.03148983 -0.81919393]
 [ 0.12337738  0.99064565 -0.048796    0.          1.          0.
   0.27321367  0.94079725 -0.04318772]
 [ 0.22737764 -0.0258913   0.86444337  0.          0.          1.
   0.5400012  -0.13358964  0.56059105]
 [ 0.74669478  0.0848657   0.49449597  0.52118275  0.12699529  0.77216728
   1.          0.          0.        ]
 [-0.10522483  1.03376256 -0.14628541 -0.17657618  1.03158439 -0.17835508
   0.          1.          0.        ]
 [-0.35149179  0.12018751  1.03617575 -0.54508206  0.12329459  0.99776283
   0.          0.          1.        ]]


In [16]:
def weaker_g(H,n,gamma,p=0.1):
    for i in range(0, n-1):
        for j in range(i+1, n):
            if np.random.rand() < p:
                H[(i)*3:(i+1)*3,(j)*3:(j+1)*3] = np.zeros([3,3])
                H[(j)*3:(j+1)*3,(i)*3:(i+1)*3] = np.zeros([3,3])
                gamma[i,j] = 0
                gamma[j,i] = 0

In [17]:
#weaker_g(H,n,gamma)
#zeta = np.sum(gamma, axis=0)

In [18]:
#print(gamma,zeta)

In [19]:
#Computation of matrix G

G = np.copy(H)
for i in range(0,n):
    G[3*i:3*(i+1),3*i:3*(i+1)] = np.eye(3,3)*(1-zeta[i])
    
np.save(os.path.join(output_dir,G_filename), G)
np.save(os.path.join(output_dir,gamma_filename), gamma)
np.save(os.path.join(output_dir,weight_matrix_filename), weight_matrix)
print(G)

[[-2.         -0.         -0.          0.8551021   0.04408762  0.32397736
   1.08865217 -0.02842227 -0.52355885]
 [-0.         -2.         -0.         -0.11780556  1.0038624   0.01289516
   0.16009339  0.9470506   0.05760714]
 [-0.         -0.         -2.         -0.22870361  0.01659748  1.07291224
   0.34991683 -0.12023143  0.78165858]
 [ 1.0769052  -0.04297181 -0.32430265 -2.         -0.         -0.
   1.05125001 -0.03148983 -0.81919393]
 [ 0.12337738  0.99064565 -0.048796   -0.         -2.         -0.
   0.27321367  0.94079725 -0.04318772]
 [ 0.22737764 -0.0258913   0.86444337 -0.         -0.         -2.
   0.5400012  -0.13358964  0.56059105]
 [ 0.74669478  0.0848657   0.49449597  0.52118275  0.12699529  0.77216728
  -2.         -0.         -0.        ]
 [-0.10522483  1.03376256 -0.14628541 -0.17657618  1.03158439 -0.17835508
  -0.         -2.         -0.        ]
 [-0.35149179  0.12018751  1.03617575 -0.54508206  0.12329459  0.99776283
  -0.         -0.         -2.        ]]


In [20]:
i=0
j=0
print("G(i,j)")
print(G[3*i:3*(i+1),3*j:3*(j+1)])
print("Gamma matrix")
print(gamma)
print("Determinants")
for m in total_matches:
    print("Source:",m.image_feature_source.index, "Destination:",m.image_feature_destination.index, "Determinant:",np.linalg.det(m.H))

G(i,j)
[[-2. -0. -0.]
 [-0. -2. -0.]
 [-0. -0. -2.]]
Gamma matrix
[[1. 1. 1.]
 [1. 1. 1.]
 [1. 1. 1.]]
Determinants
Source: 0 Destination: 1 Determinant: 0.9999999999999997
Source: 1 Destination: 0 Determinant: 0.9999999999999998
Source: 0 Destination: 2 Determinant: 0.9999999999999999
Source: 2 Destination: 0 Determinant: 0.9999999999999998
Source: 1 Destination: 2 Determinant: 1.0000000000000002
Source: 2 Destination: 1 Determinant: 1.0000000000000002


In [21]:
def buildAndPrintGraph(gamma, use_images):
    if not use_images:
        G = nx.DiGraph(gamma)
        nx.draw(G,pos=nx.circular_layout(G), node_size=900,with_labels = True, width = 1.5, edgecolors='red', node_color='lightgray')
        plt.show()
    else:
        G=nx.Graph()
        for i in range(0,n):
            G.add_node(i,image = imgs[i])
        for i in range(n): 
             for j in range(n): 
                if gamma[i,j] == 1:
                    G.add_edge(i,j) 
        pos=nx.circular_layout(G)
        fig=plt.figure(figsize=(15,15))
        ax=plt.subplot(111)
        ax.set_aspect('equal')
        nx.draw(G,pos,ax=ax, width = 3, node_size=900,with_labels = True, edgecolors='red', node_color='lightgray')
        plt.xlim(-1.5,1.5)
        plt.ylim(-1.5,1.5)
        trans=ax.transData.transform
        trans2=fig.transFigure.inverted().transform
        piesize=0.08 # this is the image size
        p2=piesize/0.9 #this is the image center
        for g in G:
            xx,yy=trans(pos[g]) # figure coordinates
            xa,ya=trans2((xx,yy)) # axes coordinates
            a = plt.axes([xa-p2,ya-p2, piesize, piesize])
            a.set_aspect('equal')
            a.imshow(G.nodes[g]['image'])
            a.axis('off')
        ax.axis('off')
        plt.show()

In [22]:
buildAndPrintGraph(gamma = gamma,use_images = True)

In [23]:
i1 = 4
i2 = 5

rescale = 1./np.max(imgs[i1].shape)
T_norm = np.diag([rescale,rescale,1])

h = G[(i2)*3:(i2+1)*3,(i1)*3:(i1+1)*3]

im_1 = cv.warpPerspective(imgs[i2], np.eye(3), [2000,1000])
im_2 = cv.warpPerspective(imgs[i1], np.linalg.inv( np.linalg.inv(T_norm)  @ h @ T_norm), [2000,1000])  #np.dot(np.dot(np.linalg.inv(T_norm),h),T_norm)

IndexError: list index out of range

In [ ]:
stitch = np.maximum(im_1, im_2)
figure(figsize=(40, 40), dpi=80)
plt.imshow(stitch[:2000,:1000],),plt.show()

In [24]:
G

array([[-2.        , -0.        , -0.        ,  0.8551021 ,  0.04408762,
         0.32397736,  1.08865217, -0.02842227, -0.52355885],
       [-0.        , -2.        , -0.        , -0.11780556,  1.0038624 ,
         0.01289516,  0.16009339,  0.9470506 ,  0.05760714],
       [-0.        , -0.        , -2.        , -0.22870361,  0.01659748,
         1.07291224,  0.34991683, -0.12023143,  0.78165858],
       [ 1.0769052 , -0.04297181, -0.32430265, -2.        , -0.        ,
        -0.        ,  1.05125001, -0.03148983, -0.81919393],
       [ 0.12337738,  0.99064565, -0.048796  , -0.        , -2.        ,
        -0.        ,  0.27321367,  0.94079725, -0.04318772],
       [ 0.22737764, -0.0258913 ,  0.86444337, -0.        , -0.        ,
        -2.        ,  0.5400012 , -0.13358964,  0.56059105],
       [ 0.74669478,  0.0848657 ,  0.49449597,  0.52118275,  0.12699529,
         0.77216728, -2.        , -0.        , -0.        ],
       [-0.10522483,  1.03376256, -0.14628541, -0.17657618,  1